In [68]:
import itertools
import numpy as np
import cvxpy as cvx

## Part A

In [69]:
# grab data
A = np.genfromtxt('data/Amatrix.csv', delimiter=',')
b = np.genfromtxt('data/bvector.csv', delimiter=',')

n, d = A.shape

verts = []
for comb in itertools.combinations(range(n), d):
    idx = np.array(comb)
    A_sub = A[idx, :]
    if np.linalg.matrix_rank(A_sub) == d:
        x_sub = np.linalg.inv(A_sub) @ b[idx]
        if np.all(A @ x_sub <= b):
            verts.append(x_sub)
    else:
        pass

print("The system has {} vertices.".format(len(verts)))
print("The vertices are:")
for vert in verts:
    print(vert)

The system has 6 vertices.
The vertices are:
[-1.32665573  1.30909998  0.15860189 -3.63057754 -0.0158668 ]
[-0.37468995  2.57136592 -0.89602599 -0.97066311 -1.16175559]
[-0.86240138 -0.18322939 -1.06390974 -1.75327847  0.71805038]
[-0.34335361 -0.66487018  0.67554263 -1.07897643  0.58301783]
[-0.60554691 -0.17010846 -1.45265679 -1.31841771 -1.17763564]
[-0.08649914 -0.65174925  0.28679557 -0.64411568 -1.31266819]


## Part B

In [70]:
# max interior ellipsoid
B = cvx.Variable((d, d), symmetric=True)
c = cvx.Variable(d)

obj = cvx.Maximize(cvx.log_det(B))
con = [cvx.norm(B @ A[i, :], 2) + A[i, :] @ c <= b[i] for i in range(n)]
prob_in = cvx.Problem(obj, con)
prob_in.solve()

print("The center of the max volume ellipsoid is: {}".format(c.value))

The center of the max volume ellipsoid is: [-1.0856306   0.99734545  0.2829785  -1.50629471 -0.57860025]


In [71]:
# lowner jon ellipsoid
K = cvx.Variable((d, d), symmetric=True)
g = cvx.Variable(d)

obj = cvx.Maximize(cvx.log_det(K))
con = [cvx.norm(K @ v + g, 2) <= 1 for v in verts]
prob_out = cvx.Problem(obj, con)
prob_out.solve(verbose=True)

print("The center of the min volume ellipsoid is: {}".format(g.value))

                                     CVXPY                                     
                                     v1.4.3                                    
(CVXPY) May 02 06:42:32 PM: Your problem has 30 variables, 6 constraints, and 0 parameters.
(CVXPY) May 02 06:42:32 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 02 06:42:32 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 02 06:42:32 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 02 06:42:32 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 02 06:42:32 PM: Compiling problem (target solver=SCS).
(CVXP

In [72]:
vol_ratio = np.exp(prob_in.value) / np.exp(prob_out.value)
print("The volume ratio is: {}".format(vol_ratio))

The volume ratio is: 0.00033377291946367846


## Part C

In [73]:
O = np.genfromtxt('data/Omatrix.csv', delimiter=',')
v = np.genfromtxt('data/vvector.csv', delimiter=',')

M = int(1e6)
x = np.random.uniform(-1,1,(d,M))

M_small = 0
M_large = 0
M_c = 0
for i in range(M):
    xp = O.T @ x[:,i] + v
    if np.all(A @ xp <= b) and np.ones(d).T @ (xp - c.value) >= 0:
        M_small += 1
    if np.all(A @ xp <= b) and np.ones(d).T @ (xp - g.value) >= 0:
        M_large += 1
    if np.all(A @ xp <= b):
        M_c += 1

In [74]:
print("R_small = {}".format(M_small / M_c))
print("R_large = {}".format(M_large / M_c))

R_small = 0.49849
R_large = 1.0
